In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
import pandas as pd

t1 = genfromtxt('train01.csv', delimiter=',')
t1 = np.delete(t1, 0, 0)
t1 = np.delete(t1, 0, 1)
x1 = t1[:,0:3]
y1 = t1[:,3]
x1[:,0] = x1[:,0]+90/180
x1[:,1] = x1[:,0]/3000
x1[:,2] = (x1[:,1])+180/360
y1 /= 60
x = x1
y = y1[:, np.newaxis] # add a newdim as granular output

LR = 0.01
batchz = 1000
epoch = 100
datapoints = len(y1)
steps = 10000

tf_x = tf.placeholder(tf.float32, x.shape)     # input x holder
tf_y = tf.placeholder(tf.float32, y.shape)     # input y holder

# Network layers
L1 = tf.layers.dense(tf_x, 10, tf.nn.tanh)
L2 = tf.layers.dense(L1, 2, tf.nn.tanh)
L3 = tf.layers.dense(L2, 2, tf.nn.tanh)
output = tf.layers.dense(L3, 1)                     # output layer

loss = tf.losses.mean_squared_error(tf_y, output)   # compute cost
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LR)
train_op = optimizer.minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(steps):
        # train and net output
        _, l, pred = sess.run([train_op, loss, output], {tf_x: x, tf_y: y})
        if step % batchz == 0:
            print(sess.run(loss, {tf_x: x, tf_y: y}))

    prediction = sess.run(output, {tf_x: x})
    time_pred = pandas.DataFrame(prediction*60)
    print(time_pred.describe())
    o_ = pandas.DataFrame({"pred": np.squeeze(prediction)*60, "ori" : y1*60})
    print("correlation")
    print(o_["pred"].corr(o_["ori"]))
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
#
# for step in range(10000):
#     # train and net output
#     _, l, pred = sess.run([train_op, loss, output], {tf_x: x, tf_y: y})
#     if step % 1000 == 0:
#         print(sess.run(loss, {tf_x: x, tf_y: y}))
#
#     prediction = sess.run(output, {tf_x: x})
#     print(prediction.shape)